In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import random
import time
import logging

In [4]:
boardgames = pd.read_csv('../data/boardgames.csv', index_col=['id'])

In [28]:
logging.basicConfig(filename='api_scraper.log', 
                    level=logging.DEBUG)

In [9]:
def calculate_pagecount(total_items):
    if int(total_items)%100 ==0:
        pages = int(total_items)//100
    else:
        pages = (int(total_items)//100)+1
    return pages

## Boardgame Classification

In [35]:
ID = 306882

In [260]:
ten_games = boardgames[boardgames['num_voters']<500].index.astype('str').tolist()[0:100]

In [210]:
ID = ','.join(ten_games)
ID

'1692,1881,3044,4493,5668,10788,19074,71593,142503,224133,254994,278120,6139,23418,93541,200834,201455,12296,233253,256390,282391,309113,3463,3828,37301,73114,145642,159566,176262,181501,183959,200456,254123,22111,24473,155702,286160,287362,116951,136117,175145,187687,221155,242088,8243,38677,108161,186987,204605,206366,233974,1098,3573,32021,58329,84838,86156,132817,152758,154910,156119,266967,10502,250561,25695,133285,179280,199877,506,6719,114871,193215,212346,234828,256669,1493,4984,53708,110870,145186,154785,177499,264806,68,6137,9439,34132,68858,125368,125755,165469,201308,257379,1413,29663,127589,132430,138311,220188,234120'

In [211]:
api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ID}&ratingcomments=1&page=6"

response = requests.get(api_adress)
response.status_code

200

In [212]:
xml = response.text

In [213]:
soup = BeautifulSoup(xml, 'xml')

In [214]:
#print(soup.prettify())

In [232]:
#<item id="1692" type="boardgame">
maxitems = 0
user_ratings = []
users = []
user_comments = []
boardgame_ids = []
ratings = pd.DataFrame()
i=0
for item in soup.find_all('item'):
    print(item.get('id'))
    categories = [link.get('value') for link in item.find_all('link', attrs={'type':'boardgamecategory'})]
    print(', '.join(categories))

    page = int(item.find('comments').get('page'))
    total_items = int(item.find('comments').get('totalitems'))
    if total_items > maxitems:
        maxitems = total_items
    #comments = item.find('comments')
    for comment in item.find_all('comment'):
        #print(i)
        boardgame_ids.append(item.get('id'))
        user_ratings.append(comment.get('rating'))
        users.append(comment.get('username'))
        user_comments.append(comment.get('value')) 
        i = i+1
    print(item.get('id'),page, total_items, len(user_comments))
 

1692
Action / Dexterity, Animals, Card Game, Children's Game, Real-time
1692 6 502 2
1881
Dice, Fantasy, Maze, Racing
1881 6 498 2
3044
Aviation / Flight, Wargame, World War II
3044 6 502 4
4493
Action / Dexterity, Children's Game
4493 6 500 4
5668
Abstract Strategy, Deduction
5668 6 506 10
10788
Ancient, Wargame
10788 6 499 10
19074
Ancient, Economic
19074 6 499 10
71593
Science Fiction
71593 6 502 12
142503
Bluffing, Card Game, Deduction, Negotiation, Party Game
142503 6 502 14
224133
Card Game, Dice, Medieval, Renaissance
224133 6 505 19
254994
Nautical, Pirates, Racing
254994 6 503 22
278120
Video Game Theme
278120 6 508 30
6139
Adventure, Civilization, Expansion for Base-game, Fantasy, Fighting
6139 6 500 30
23418
Arabian, Wargame, World War I
23418 6 500 30
93541
City Building
93541 6 502 32
200834
Card Game, Educational, Political, Religious
200834 6 498 32
201455
Fantasy, Miniatures
201455 6 499 32
12296
Ancient, Card Game, Political
12296 6 498 32
233253
Adventure, Expansion f

In [216]:
   
ratings = pd.DataFrame({
    'boardgame_id':boardgame_ids,
    'user_name': users,
    'rating': user_ratings,
    'comment': user_comments
})

In [218]:
ratings.head(30)

,boardgame_id,user_name,rating,comment
0,1692,OGRussHood,1,
1,1692,mmychal,1,
2,3044,Mundster,2,The best thing about this game was the Malta m...
3,3044,rcrumrine1,2,Should have been good but the system is too wo...
4,5668,Baartoszz,2,barely a game
5,5668,Zombie Killer,2,
6,5668,steroo,2,
7,5668,GrzeL,2,
8,5668,pdowen3,2,
9,5668,NeutralGnome,2,


In [130]:
ratings.loc[190:210]

,boardgame_id,user_name,rating,comment
190,1881,Yatterman,7.5,
191,1881,DamienMocata,7.5,Another one from childhood here. Same disclaim...
192,1881,Fiend138,7.5,
193,1881,blanetalk,7.5,
194,1881,Brad_wh1tt,7.5,
195,1881,Bassplant,7.4,
196,1881,Briareos,7,
197,1881,Nojobro,7,Great fun if you don't play very often.
198,1881,rampantdragon,7,
199,1881,reapersaurus,7,Lots of fun - quite a few couples like this at...


In [156]:
maxitems

574

In [227]:
boardgames.index.tolist().index(1692)

5449

In [225]:
boardgames.index[5449]

1692

In [182]:
boardgame_list = boardgames.index.tolist()
start_id = 1692
start_index = boardgame_list.index(start_id)
end_index = start_index + 
end_id = boardgame_list[end_index]

print(start_index, end_index)
print(start_id, end_id)

5449 5549
1692 298572


In [202]:
step_width = 100
start_index = 23
end_index = start_index + step_width
boardgame_list = boardgames.index[start_index:end_index].astype('str').tolist()
boardgame_list

['266192',
 '28143',
 '110327',
 '34635',
 '174430',
 '161936',
 '65244',
 '1927',
 '12333',
 '120677',
 '150376',
 '98778',
 '11',
 '54043',
 '50',
 '10547',
 '15987',
 '131357',
 '147020',
 '41114',
 '70919',
 '133473',
 '121921',
 '170216',
 '204583',
 '37111',
 '181304',
 '463',
 '157354',
 '126163',
 '183394',
 '193738',
 '181',
 '40834',
 '2655',
 '157969',
 '205637',
 '146021',
 '31481',
 '1406',
 '171',
 '201808',
 '124361',
 '194655',
 '6249',
 '103343',
 '102794',
 '96848',
 '162886',
 '39463',
 '128882',
 '199561',
 '320',
 '132531',
 '237182',
 '124742',
 '18602',
 '143884',
 '35677',
 '3955',
 '72125',
 '205059',
 '9220',
 '136063',
 '10630',
 '148949',
 '15062',
 '42',
 '158899',
 '244521',
 '172',
 '146508',
 '182028',
 '122522',
 '187645',
 '93',
 '50381',
 '16992',
 '192291',
 '199792',
 '103885',
 '2453',
 '153938',
 '164928',
 '104162',
 '172225',
 '18',
 '155426',
 '100423',
 '8217',
 '147949',
 '2223',
 '123260',
 '224517',
 '154203',
 '432',
 '100901',
 '108745',


In [203]:
','.join(boardgame_list)

'266192,28143,110327,34635,174430,161936,65244,1927,12333,120677,150376,98778,11,54043,50,10547,15987,131357,147020,41114,70919,133473,121921,170216,204583,37111,181304,463,157354,126163,183394,193738,181,40834,2655,157969,205637,146021,31481,1406,171,201808,124361,194655,6249,103343,102794,96848,162886,39463,128882,199561,320,132531,237182,124742,18602,143884,35677,3955,72125,205059,9220,136063,10630,148949,15062,42,158899,244521,172,146508,182028,122522,187645,93,50381,16992,192291,199792,103885,2453,153938,164928,104162,172225,18,155426,100423,8217,147949,2223,123260,224517,154203,432,100901,108745,77423,123540'

In [185]:
len(boardgames.index)

45000

In [219]:
step_width = 100
start_index = 23
end_index = start_index + step_width

while start_index < len(boardgames.index):
    #print(boardgames.index[start_index:end_index])
    #print(len(boardgames.index[start_index:end_index]))
    start_index = end_index
    end_index = start_index + step_width

In [220]:
for i in range(2,6):
    print(i)

2
3
4
5


In [189]:
boardgames.tail()

,rank,name,links,num_voters
id,,,,
138578,NaN,Silent But Deadly,/boardgame/138578/silent-deadly,10
138730,NaN,Hull Breach: In Defiance of Dictators,/boardgame/138730/hull-breach-defiance-dictators,10
139581,NaN,Wahrheit oder Blödsinn?,/boardgame/139581/wahrheit-oder-blodsinn,10
139637,NaN,Zombie Shuffle,/boardgame/139637/zombie-shuffle,10
139930,NaN,Flush!,/boardgame/139930/flush,10


In [ ]:
categories = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamecategory'})]
categories

In [ ]:
mechanics = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamemechanic'})]
mechanics

In [ ]:
family = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamefamily'})]
family

In [ ]:
expansions = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameexpansion'})]
expansions

In [ ]:
integrations = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameintegration'})]
integrations

In [ ]:
designers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamedesigner'})]
designers

In [ ]:
publishers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamepublisher'})]
publishers 

## User Ratings

In [ ]:
api_adress = "https://www.boardgamegeek.com/xmlapi2/thing?id=306882&ratingcomments=1&page=1"

response = requests.get(api_adress)
response.status_code

In [ ]:
xml = response.text
soup = BeautifulSoup(xml, 'xml')

In [ ]:
#print(soup.prettify())

In [ ]:
page = soup.find('comments').get('page')
page

In [ ]:
total_items = soup.find('comments').get('totalitems')
total_items

In [ ]:
pagecount = calculate_pagecount(total_items)
pagecount

In [ ]:
ratings = []
users = []
comments = []
for comment in soup.find_all('comment'):
    ratings.append(comment.get('rating'))
    users.append(comment.get('username'))
    comments.append(comment.get('value'))
    

In [ ]:
boardgame_id = [ID] * len(ratings)

In [ ]:
ratings = pd.DataFrame({
    'id':boardgame_id,
    'user':users,
    'rating':ratings,
    'comment':comments
})

In [ ]:
ratings['rating'].isna().sum()

To DO:
- Loop through pages
- ignore users without ratings
- User IDs?
- include logging
- append csv not dataframe
- check request status

## Loop over rating pages

In [29]:
boardgames =pd.read_csv('../data/boardgames.csv', index_col='id')
boardgames['categories'] = None
boardgames['mechanics'] = None
boardgames['family'] = None
boardgames['expansions'] = None
boardgames['integrations'] = None
boardgames['designers'] = None
boardgames['publishers'] = None

sleep_default = 2

In [34]:
for ID in [4974, 6927, 8139, 8993, 13121]:
    logging.debug(f'movie id: {ID}')
    
    user_ratings = []
    users = []
    comments = []
    ratings = pd.DataFrame()
    page = 1
    
    # request page 1 of game overview with user ratings and comments
    api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ID}&ratingcomments=1&page={page}"
    response = requests.get(api_adress)  
    logging.info(f'status {response.status_code} for {api_adress}')    
    
    xml = response.text
    soup = BeautifulSoup(xml, 'xml')
    
    # get the number of pages of ratings
    total_items = soup.find('comments').get('totalitems')
    logging.debug(f'total items: {total_items}')
    pagecount = calculate_pagecount(total_items)
    logging.debug(f'pagecount: {pagecount}')
    logging.debug(f'start scraping page: {page}')

    
    # extend boardgames dataframe 
    categories = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamecategory'})]
    boardgames.loc[ID,'categories'] = ', '.join(categories)
    mechanics = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamemechanic'})]
    boardgames.loc[ID,'mechanics'] = ', '.join(mechanics)
    family = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamefamily'})]
    boardgames.loc[ID,'family'] = ', '.join(family)
    expansions = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameexpansion'})]
    boardgames.loc[ID,'expansions'] = ', '.join(expansions)
    integrations = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameintegration'})]
    boardgames.loc[ID,'integrations'] = ', '.join(integrations)
    designers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamedesigner'})]
    boardgames.loc[ID,'designers'] = ', '.join(designers)
    publishers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamepublisher'})]
    boardgames.loc[ID,'publishers'] = ', '.join(publishers)
    
    # update number of voters
    boardgames.loc[ID, 'num_voters'] = total_items
    
    boardgames.to_csv('../data/boardgames_extend.csv', index=False)
    
    #get user ratings from page 1
    for comment in soup.find_all('comment'):
        user_ratings.append(comment.get('rating'))
        users.append(comment.get('username'))
        comments.append(comment.get('value')) 
        
    boardgame_id = [ID] * len(user_ratings)

    ratings = pd.DataFrame({
        'id':boardgame_id,
        'user':users,
        'rating':user_ratings,
        'comment':comments
    })
        
    ratings.to_csv('../data/ratings.csv', index=False, header=None, mode='a')
    
    # if more than one page of comments send request for each page
    if pagecount > page:
        page = page + 1

        while page <= pagecount:
            user_ratings = []
            users = []
            comments = []
            ratings = pd.DataFrame()
            
            api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ID}&ratingcomments=1&page={page}"
            response = requests.get(api_adress)
            logging.info(f'status {response.status_code} for {api_adress}')
            
            if response.status_code != 200:
                logging.warning(f'STATUS {response.status_code}, increase sleep time and try again')
                time.sleep(10)
                sleep_default = 5
                
            else:
            
                xml = response.text
                soup = BeautifulSoup(xml, 'xml')

                for comment in soup.find_all('comment'):
                    user_ratings.append(comment.get('rating'))
                    users.append(comment.get('username'))
                    comments.append(comment.get('value'))

                boardgame_id = [ID] * len(user_ratings)

                ratings = pd.DataFrame({
                    'id':boardgame_id,
                    'user':users,
                    'rating':user_ratings,
                    'comment':comments
                })

                ratings.to_csv('../data/ratings.csv', index=False, header=False, mode='a')

                page = page + 1
                time.sleep(sleep_default+random.random())


In [247]:
ratings2 = pd.read_csv('../data/ratings_2.0.csv', names=['boardgame_id', 'user_name', 'rating', 'comment'])
ratings2

,boardgame_id,user_name,rating,comment
0,1692,oneofeight8,10.0,i love this game it is so fun
1,1692,Dale-not-Chip,10.0,Can't help it. It's just fun.
2,1692,teddyweiche,10.0,NaN
3,1692,GameTutor,10.0,Have been playing this game all my life. It n...
4,1692,KidDarlinDragonfist,10.0,NaN
...,...,...,...,...
6010,93541,altair79,7.0,10
6011,93541,mandi_lea,7.0,NaN
6012,93541,affm,7.0,NaN
6013,93541,oowick,7.0,Solid early entry from Luciani; much more cons...


In [248]:
ratings2[ratings2['boardgame_id']==1692].iloc[190:210]

,boardgame_id,user_name,rating,comment
590,1692,Sobo,6.0,NaN
591,1692,DocGerbil,6.0,The game of masochism.
592,1692,bponnaluri,6.0,NaN
593,1692,cybulskina,6.0,NaN
594,1692,GMGDave,6.0,NaN
595,1692,teal,6.0,NaN
596,1692,Simsmzzms,6.0,NaN
597,1692,berkstrom,6.0,NaN
598,1692,berkstrom,6.0,NaN
599,1692,Eggers,6.0,NaN


In [252]:
ratings2.iloc[2490:2510]

,boardgame_id,user_name,rating,comment
2490,5668,Gosunkugi,3.0,NaN
2491,5668,rockwoofstone,3.0,NaN
2492,5668,HilariousDwarf,3.0,NaN
2493,5668,I am nobody,3.0,NaN
2494,5668,oldgoat3769967,3.0,NaN
2495,5668,Lexi,2.0,NaN
2496,5668,badweasel,2.0,NaN
2497,5668,Kaffedrake,2.0,Not an improvement on the original Mastermind....
2498,1692,OGRussHood,1.0,NaN
2499,1692,mmychal,1.0,NaN


In [253]:
boardgames2 = pd.read_csv('../data/boardgames_extend_2.0.csv')

In [255]:
boardgames2.iloc[5449]

rank                                                        20587
name                                                       Spoons
links                                      /boardgame/1692/spoons
num_voters                                                    502
categories      Action / Dexterity, Animals, Card Game, Childr...
mechanics           Set Collection, Simultaneous Action Selection
family          Card Games: Exchange, Card Games: Matching, Ca...
expansions                                                    NaN
integrations                                                  NaN
designers                                           Michael Ferch
publishers      (Public Domain), Asmodee, Crown & Andrews Ltd....
Name: 5449, dtype: object

In [261]:
boardgames.loc[119788:]

,rank,name,links,num_voters
id,,,,
119788,697.0,Hyperborea,/boardgame/119788/hyperborea,4456
150145,522.0,Skull King,/boardgame/150145/skull-king,4455
233020,1094.0,Fireball Island: The Curse of Vul-Kar,/boardgame/233020/fireball-island-curse-vul-kar,4451
35815,1025.0,A Touch of Evil: The Supernatural Game,/boardgame/35815/touch-evil-supernatural-game,4449
30957,660.0,Tribune: Primus Inter Pares,/boardgame/30957/tribune-primus-inter-pares,4447
...,...,...,...,...
138578,NaN,Silent But Deadly,/boardgame/138578/silent-deadly,10
138730,NaN,Hull Breach: In Defiance of Dictators,/boardgame/138730/hull-breach-defiance-dictators,10
139581,NaN,Wahrheit oder Blödsinn?,/boardgame/139581/wahrheit-oder-blodsinn,10
